# 실습 [7-1]<br>
**실습명: Lesk 알고리즘을 이용한 단어 중의성 해소**
- lesk 알고리즘: WordNet의 사전 정의 정보를 이용해 중의성 단어와 맥락의 pattern 찾는 알고리즘

In [1]:
#WordNet 관련 패키지 import
import nltk

nltk.download('wordnet')
nltk.download('punkt') #punctuation
nltk.download('stopwords') #불용어

from nltk.corpus import wordnet
from nltk import word_tokenize
from nltk.corpus import stopwords
import sys

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
#단어&문장에 나타난 단어에 대해 best sense(의미) 추출
def disambiguate(word, sentence, stopwords):
  word_senses = wordnet.synsets(word) #단어 의미에 대한 후보 모두 불러오기
  best_sense = word_senses[0] #가장 빈도수가 높은 단어가 best sense
  max_overlap = 0 #겹치는 것 제외
  context = set(word_tokenize(sentence))
  for sense in word_senses:
      signature = tokenized_gloss(sense)
      overlap = compute_overlap(signature, context, stopwords)
      if overlap > max_overlap:
          max_overlap = overlap
          best_sense = sense
  return best_sense

In [3]:
#sense(의미)의 definition에 대한 모든 token 추출
def tokenized_gloss(sense):
  tokens = set(word_tokenize(sense.definition()))
  for example in sense.examples():
    tokens.union(set(word_tokenize(example)))
  return tokens

In [4]:
#겹치는 단어 비교
def compute_overlap(signature, context, stopwords):
  gloss = signature.difference(stopwords)
  return len(gloss.intersection(context))

In [5]:
#NLTK에서 지정한 영어 불용어 처리
stopwords = set(stopwords.words('english'))
sentence = ("They eat a meal") #중의적 단어가 들어있는 문장
context = set(word_tokenize(sentence)) #주변 맥락 (문장의 단어열들 모두 토큰화)
word = 'eat' #중의적 단어

In [6]:
print("Word : ", word)
syn = wordnet.synsets('eat')[1]
print("Sense : ", syn.name()) #중의적 단어 의미
print("Defintion : ", syn.definition()) #중의적 단어 definition
print("Sentence : ", sentence)

Word :  eat
Sense :  eat.v.02
Defintion :  eat a meal; take a meal
Sentence :  They eat a meal


In [8]:
signature = tokenized_gloss(syn) #모든 단어에 대한 토큰화
print("Tokenized words : ", signature)
print("Overlap count : ", compute_overlap(signature, context, stopwords))
print("Best sense : ", disambiguate(word, sentence, stopwords))

Tokenized words :  {'take', 'a', 'eat', ';', 'meal'}
Overlap count :  2
Best sense :  Synset('eat.v.02')
